# Lecture 10: Sparse Coding, Dictionary Learning

In the previous lecture, we talked about different ways to encode a signal in a sparse way using either an orthonormal basis dictionary, or an overcomplete dictionary.

We saw the tradeoffs between orthogonal dictionaries (easy coding and unique reconstruction, but sparsity limited to a certain class of data) and overcomplete ones (harder to find sparser coding accurately, but able to sparsely encode more classes of signals). These were both treated in the case of **fixed bases**.

It was mentioned that a basis could be either fixed in advanced (e.g. Haar Wavelet basis, Fourier basis), or computed optimally based on a dataset (i.e. several separate signals each needing compression). The latter version, however, was not elaborated on. We will do that in the second part of this lecture.

Overview for this lecture:
 * Matching Pursuit algorithm for finding a sparse coding
 * Learning an optimal dictionary $U$ from a dataset

### Short aside: DCT vs. DFT

> The DFT uses a set of harmonically-related complex exponential functions, while the DCT uses only (real-valued) cosine functions.
> The DFT is widely used for general spectral analysis applications that find their way into a range of fields. It is also used as a building block for techniques that take advantage of properties of signals' frequency-domain representation, such as the overlap-save and overlap-add fast convolution algorithms.
> The DCT is frequently used in lossy data compression applications, such as the JPEG image format. The property of the DCT that makes it quite suitable for compression is its **high degree of "spectral compaction;"** at a qualitative level, a signal's DCT representation tends to have more of its energy concentrated in a small number of coefficients when compared to other transforms like the DFT. This is desirable for a compression algorithm; if you can approximately represent the original (time- or spatial-domain) signal using a relatively small set of DCT coefficients, then you can reduce your data storage requirement by only storing the DCT outputs that contain significant amounts of energy.

Source: https://dsp.stackexchange.com/questions/13/what-is-the-difference-between-a-fourier-transform-and-a-cosine-transform (Answer by person focused on signal processing)

Summary:
 * The DCT is capable of encoding more information (energy) into fewer coefficients, so it's suitable for compression.
 * The DFT is more useful for e.g. analysis.
 * The DFT uses complex exponential functions, while the DCT uses only real cosine functions.

## Matching Pursuit
 * Iterative greedy algorithm.
 * Approximates an NP-hard problem.
 * Generally: 
  - start with coding vector $z = 0$ and residual $r_0 = x$
  - at every step, find the atom $\mathbf{u}_{i^*}$ which minimizes the residual $r = \| x - U z \|_2$

 
### Atom selection

Want:

$$
i^{\star} \in \underset{i}{\arg\!\max} \left| \langle r^{t-1}, u_i \rangle \right|
$$

I.e. the atom $u_i$ which "has the most in common" with the residual.

Proof that picking that $i^{\star}$ also minimizes the residual in the next step (which is what we want!):

$$
\alpha_i^{\star} = \underset{\alpha}{\arg\!\min} \| r - \alpha u_i \|_2^2
\iff
\alpha_i^{\star} = \langle r, u_i \rangle, \quad \forall u_i, \>\|u_i\|_2 =1
$$

The above follows from the definition of the inner product of two vectors, and its geometric interpretation: it's the length of the projection of one of the vectors onto the other. The optimal scaling coefficient of the unit length vector u is the magnitude of r's projection on it, i.e. $\langle r, u_i \rangle$.

The residual update equation $r_i^t := r^{t-1} - \alpha u_i$ becomes:

$$
r_i^t := r^{t-1} - \langle r^{t-1}, u_{i} \rangle u_i
$$

This is equivalent to:

$$
r^{t-1} = r_i^t + \langle r^{t-1}, u_{i} \rangle u_i = r_i^t + (r^{t-1})^T u_i u_i
$$

Warning, steps may not be 100% correct! Also, mixing $a^Tb$ and $\langle a, b\rangle$ notation for the inner product is not ideal, but can help simplify the calculations a little.

$$
\| r^{t-1} \|^2 = \| r_i^t \|^2 + (\langle r^{t-1}, u_{i} \rangle u_i)^T (\langle r^{t-1}, u_{i} \rangle u_i) + 2 (r_i^t)^T (\langle r^{t-1}, u_{i}\rangle u_i )
$$

We can now work to get rid of the final term. We take the inner product from the middle out, since it's a constant:

$$
\| r^{t-1} \|^2 = \| r_i^t \|^2 + (\langle r^{t-1}, u_{i} \rangle u_i)^T (\langle r^{t-1}, u_{i} \rangle u_i) + 2 \langle r^{t-1}, u_{i}\rangle (r_i^t)^T (u_i)
$$

Since we just removed all the $u_i$ from $r_i^t$ (its definition), they're now orthogonal, so $\langle r_i^t, u_i \rangle = 0$. We can therefore drop the final term:

$$
\| r^{t-1} \|^2 = \| r_i^t \|^2 + (\langle r^{t-1}, u_{i} \rangle u_i)^T (\langle r^{t-1}, u_{i} \rangle u_i) + 0
$$

We now reorganize the second term so that the inner products in the middle get moved to the front of the term, since they're both just real numbers (working purely with row/column vector multiplications leads to the same result, only over a few extra steps):

$$
\| r^{t-1} \|^2 = \| r_i^t \|^2 + \langle r^{t-1}, u_{i} \rangle \langle r^{t-1}, u_{i} \rangle u_i^T u_i
$$

We can compress things even further and drop the final $u_i^T u_i = \| u_i \|^2$ norm, since all atoms are assumed to be unit-length:

$$
\underbrace{\| r^{t-1} \|^2}_{\text{old residual, const.}} =
\underbrace{\| r_i^t \|^2}_{\text{minimize}} + 
\underbrace{\langle r^{t-1}, u_{i} \rangle^2}_{\text{maximize}}
$$

We now showed that maximizing one term is equivalent to minimizing the other.

I.e. **minimizing the new residual is equivalent to picking a new atom with the highest inner product to the previous residual. This is how (and why) Matching Pursuit works.**


### The algorithm

#### Objective

$$
z^{\star} \in {\arg\!\min}_z \| x - Uz \|_2, \quad \text{s.t.} \> \|z\|_0 < K
$$

#### Steps

 1. $z \gets 0, \> r \gets x$
 2. **while** $ \| z \|_0 < K $ **do**:
     3. Find the best atom: $i^{\star} \gets {\arg\!\max}_i \langle u_i, r \rangle  $ 
     4. Update the coding: $z_{i^\star} \gets z_{i^\star} + \langle u_i, r \rangle $ 
     5. Update the residual: $ r \gets r - \langle u_{i^\star}, r \rangle u_{i^\star} $

### Success of MP

When is the greedy solution exact?

If dictionary is overcomplete and we want a not-so-sparse coding (e.g. D = 10, want card of coding 8-10), then it's problematic. But if the true coding is sparse enough, we're OK, even with an overcomplete dictionary.


Theory: exact recovery condition for Matching Pursuit:

$$
K < \frac{1}{2}\left( 1 + \frac{1}{m(U)} \right)
$$

Where $m(U) = \max_{i\not=j} \| u_i^T u_j \| $ is the **dictionary's coherence**.

So if the coherence is low enough, and K is lower than the bound, **MP recovers true support**.

## Compressive Sensing

Motivation: instead of capturing "raw" data and compressing it right away anyway, try to capture compressed data directly! (Useful e.g. for space imaging aboard low-power probes, or during MRI to reduce scanning time).

$$ x = Uz, \quad \text{s.t.} \> \| z \|_0 = K $$

The real signal in which we're interested is $x$, and we know in advance that it's K-sparse in some orthonormal basis $U$.

Capture several measurements $y_k$, $k = 1, \ldots, M$:

$$
y = Wx = WUz =: \Theta z, \quad \Theta = WU \in \mathbb{R}^{M \times D}
$$

Want x. Have y (measurements), W (randomly generated; see below), and U, known.

**Cannot compute x directly!**

 1. Find sparsest z: $$z^{\star} \in {\arg\!\min}_z \| z \|_0, \> \text{s.t.} y = \Theta z $$using the techniques presented so far, e.g. MP
 2. $x = Uz$ (this is the easy part!)


### Conditions

Q: How do we pick the weights in W?

A: Gaussian **random** projection! $w_{i,j} \sim \mathcal{N}\left( 0, \frac{1}{D} \right)$

Q: How many measurements (M) do we need?

A: $M \ge c K \log\left(\frac{D}{K}\right)$, where $c$ is some constant.


We therefore see that for compressive sensing, the sparse coding part is actually not the whole thing, like it would be in, say, a plain signal compression problem, but part of a longer pipeline.


## Dictionary Learning

How to adapt a dictionary $U$ to a learning problem?

Fixed orthonormal basis:
 * Pros: pre-existing, efficient coding ($z = U^T x$)--it's a snap!
 * Cons: only works well for a very specific fixed class of signals, otherwise no longer sparse.
 
Fixed overcomplete basis:
 * Pros: pre-existing, covers several signal classes.
 * Cons: may need approximation algorithm to find sparsest code, since multiple solutions can now exist.
 
Learned dictionary:
 * Pros: Guaranteed sparsity on current dataset (same approximation error with smaller dictionary size L)
 * Cons: Have so solve a matrix factorization problem to compute dictionary U dynamically. Still may need approx. algorithms to find sparsest code given x and U.

### Matrix Factorization for Dictionary Learning

Want:

$$
(\mathbf{U}^\star, \mathbf{Z}^\star) \in \underset{\mathbf{U}, \mathbf{Z}}{\arg\!\min}
\| X - UZ \|^2_F
$$

Objective **not jointly convex** in U and Z, but convex in **either U or Z** $\implies$ use alternating iterative minimization.

 1. Find best coding Z for fixed dictionary U.
 $$
 Z^{t+1} \in {\arg\!\min}_Z \| X - U^t Z \|^2_F \quad \text{s.t.} \> Z \text{ - sparse}
 $$
 2. Find best dictionary U for fixed coding Z.
 $$
 U^{t+1} \in {\arg\!\min}_U \| X - U Z^{t+1} \|^2_F \quad \text{s.t.} \> \| u_l \|_2 = 1 , \, \forall l
 $$

#### Coding step

Since we're working with column-wise sparsity constraints, and squared Frobenius norm, we can separate the residual for each column.

**N independent** sparse coding steps for every column $n$.

$$
\begin{aligned}
z_n^{t+1}  & \in & {\arg\!\min}_z \|z\|_0 \\
\text{s.t.} && \| x_n - U^t z_n \| \le \sigma \| x_n \|_2
\end{aligned}
$$

TODO(andrei): What is this sigma? Why does the error constraint depend on the magnitude of the signal?

#### Dictionary update step

Sadly, **not separable column (atom)-wise**. But we still update one atom at a time! It's an approximation, but it works.

Performed using SVD and picking $u_l^*$ as the first left-singular vector:

![K-SVD dictionary update](img/dictionary-learning.png)